In [135]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

dfPolicyData = pd.read_csv("PolicyData.csv", delimiter = ";", encoding='latin-1')

dfInvoiceData = pd.read_csv("InvoiceData.csv", delimiter = ";")

In [136]:
# 1: Invoice is paid late
# 0: Invoice is not paid late
aLabels = [1, 0]
aConditions = [
    (dfInvoiceData["due_date"] < dfInvoiceData["paid_date"]),
    (dfInvoiceData["due_date"] >= dfInvoiceData["paid_date"])
]

dfInvoiceData["label"] = np.select(aConditions, aLabels)

In [137]:
#they are converted from object to float
dfInvoiceData["amount_premium"] = dfInvoiceData["amount_premium"].apply(lambda x: x.replace(',','.'), ).astype(float, errors = 'raise')
dfPolicyData["Premium"] = dfPolicyData["Premium"].apply(lambda x: x.replace(',','.'), ).astype(float, errors = 'raise')

dfPolicyData.info()
dfInvoiceData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68638 entries, 0 to 68637
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   policy_guid                   68638 non-null  object 
 1   Country                       68638 non-null  object 
 2   VehicleType                   68638 non-null  object 
 3   VehicleUsage                  68638 non-null  object 
 4   Power                         68638 non-null  int64  
 5   Weight                        68638 non-null  int64  
 6   VehicleFirstRegistrationYear  68638 non-null  int64  
 7   Mark                          68638 non-null  object 
 8   Model                         68638 non-null  object 
 9   Deductible_general            68636 non-null  float64
 10  Fire                          68638 non-null  int64  
 11  Theft                         68638 non-null  int64  
 12  Natural_disasters             68638 non-null  int64  
 13  R

In [138]:
# there are 12 types of number of payments
np.sort(dfPolicyData["Nb_of_payments"].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [139]:
dfInvoiceData[["policy_guid", "label"]].groupby(["policy_guid"]).agg(['sum', 'count'])

label      
                                         sum count
policy_guid                                       
{00002943-8C7A-4293-9715-0E4F09C6311D}     0     1
{000118B2-9E9C-4A98-A338-DC31DD7C1F59}     1    12
{0001A384-9AAC-44DF-BE77-D247F880B80B}     1     1
{0001C1D4-FB9E-4DD3-A539-1F53C3E2BDD9}     1     1
{00027C87-4647-41EB-A406-6EB60D5D83D8}     0    12
...                                      ...   ...
{FFFD2CDF-04DD-4DDE-9896-B4E7E1614847}     4     6
{FFFE2A08-A419-42B9-96A7-A71BF9BEC562}     2     6
{FFFF00E5-717D-4E62-B39A-E219EDD47E09}     2    12
{FFFF8151-3C7B-46F9-B971-7D5F03DB3B52}     0     1
{FFFFCA48-BC08-0C4B-AAB9-FA172CEB764C}     0     2

[68638 rows x 2 columns]

In [140]:
#In the dataset not all policies are ended. It s a snapshot probably around June 2018.
#There could be 3 status of policies: "Ended on time", "Terminated" and "Ongoing"
#We can build model not based on policy status, but based on number of issued invoices.

In [163]:
#Missing data is represented either with 'Missing' or NA.

dfPolicyData[dfPolicyData.isnull().any(axis=1)]

dfMissing = dfPolicyData.astype(str) == "Missing"
dfPolicyData[dfMissing.any(axis=1)]

,policy_guid,Country,VehicleType,VehicleUsage,Power,Weight,VehicleFirstRegistrationYear,Mark,Model,Deductible_general,Fire,Theft,Natural_disasters,Road_assistance,Total_loss,Vandalism,Glass,Accident,Replacement_car,Gender,ClientBirthday,Region,BMClassMOD,Leasing,PolicyIssueDate,PolicyStartDate,PolicyEndDate,PolicyActualEndDate,Nb_of_payments,Premium,Channel,FuelType,avgFuelConsumption,DriveTrain,RenewalIndicator,IsRenewed,BalticRating,Terminated,sales_type
3,{74A1885E-CC4D-435B-B7CF-CAD0287FA814},EE,Passenger car,Regular,85,2510,2007,VOLKSWAGEN,SHARAN,191.0,1,1,1,1,1,1,1,1,0,M,1954-03-08 00:00,TARTUMAA,A1,0,2015-09-20 00:00,2015-09-20 00:00,2016-09-19 00:00,2015-10-04 00:00,12,275.24,Direct,Missing,NaN,Missing,0,1,5,1,New sale
4,{FF6D78C8-B660-4D58-8A9D-0FD6E32E2A03},EE,Passenger car,Regular,77,1780,2013,SKODA,OCTAVIA,191.0,1,0,1,1,1,1,1,1,0,M,1944-07-30 00:00,JÕGEVAMAA,0,0,2015-11-23 00:00,2015-11-23 00:00,2016-11-22 00:00,2016-04-22 00:00,12,239.77,Direct,Gasoline,NaN,Missing,0,0,2,1,New sale
6,{77540A90-4A76-4555-945A-D9EFC8BEF959},EE,Passenger car,Regular,120,2590,2005,VOLVO,XC90,191.0,1,1,1,1,1,1,1,1,0,M,1959-05-31 00:00,IDA-VIRUMAA,0,0,2015-09-26 00:00,2015-09-26 00:00,2016-09-25 00:00,2015-11-05 00:00,4,477.43,Direct,Diesel,NaN,Missing,0,0,3,1,New sale
7,{10BF0501-6360-4AF2-BB1E-CBB4183EE432},EE,Passenger car,Regular,103,2505,2007,FORD,S-MAX,191.0,1,1,1,1,1,1,1,1,0,F,1973-05-15 00:00,PÄRNUMAA,0,0,2015-09-26 00:00,2015-09-26 00:00,2016-09-25 00:00,2016-09-25 00:00,4,245.00,Direct,Diesel,NaN,Missing,0,1,0,0,New sale
8,{D57F06CB-31EA-42F6-8228-39E9DE5A3DF5},EE,Passenger car,Regular,103,2140,2006,HONDA,CR V,191.0,1,0,1,1,1,1,1,1,0,M,1960-10-10 00:00,LÄÄNE-VIRUMAA,0,0,2015-09-15 00:00,2015-09-15 00:00,2016-09-14 00:00,2016-09-14 00:00,12,265.23,Direct,Diesel,NaN,Missing,0,1,3,0,New sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68622,{3F9628B8-AD1B-4716-86B1-9B696AC23767},EE,Passenger car,Regular,124,2050,2008,NISSAN,XTRAIL,200.0,0,0,0,0,0,0,0,0,0,M,1942-07-11 00:00,HARJUMAA,A4,0,2017-06-16 00:00,2017-06-16 00:00,2018-06-15 00:00,2018-06-15 00:00,4,525.60,Direct,Gasoline,NaN,Missing,1,0,4,0,Renewal
68626,{92214079-F3D2-43CF-80E7-1CA77372D087},EE,Passenger car,Regular,110,1990,2005,HONDA,CR V,200.0,0,0,0,0,0,0,0,0,0,M,1963-05-18 00:00,PÕLVAMAA,A2,0,2017-08-14 00:00,2017-08-14 00:00,2018-08-13 00:00,2018-08-13 00:00,2,319.48,Direct,Gasoline,NaN,Missing,1,0,4,0,Renewal
68627,{A38D64CE-810B-4692-A868-CD16B897FA31},EE,Tractor,ATV,36,470,2007,OTHER,LT-A700XK7,400.0,1,1,0,0,0,0,0,0,0,M,1979-08-16 00:00,JÕGEVAMAA,A2,0,2017-07-30 00:00,2017-07-30 00:00,2018-07-29 00:00,2018-07-29 00:00,4,110.01,Direct,Gasoline,NaN,Missing,1,0,2,0,Renewal
68636,{CB6A2E29-CB75-4893-979F-CE844C970745},EE,Van,Regular,94,3500,2006,FIAT,DUCATO,200.0,0,0,0,0,0,0,0,0,0,M,1979-02-12 00:00,VILJANDIMAA,A4,0,2018-03-11 00:00,2018-03-11 00:00,2019-03-10 00:00,2019-03-10 00:00,4,356.70,Direct,Diesel,NaN,Missing,1,0,4,0,Renewal
